<a href="https://colab.research.google.com/github/abelshinev/baggage-threat-scanner/blob/main/PIDray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Testing mount

!ls /content/drive/MyDrive/Datasets

OPIXRay_YOLO  PIDray_raw


In [ ]:
!unzip /content/drive/MyDrive/Datasets/PIDray_raw/train.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/xray_71913.png    
  inflating: train/xray_71914.png    
  inflating: train/xray_71915.png    
  inflating: train/xray_71916.png    
  inflating: train/xray_71917.png    
  inflating: train/xray_71918.png    
  inflating: train/xray_71919.png    
  inflating: train/xray_71920.png    
  inflating: train/xray_71921.png    
  inflating: train/xray_71922.png    
  inflating: train/xray_71923.png    
  inflating: train/xray_71924.png    
  inflating: train/xray_71925.png    
  inflating: train/xray_71926.png    
  inflating: train/xray_71927.png    
  inflating: train/xray_71928.png    
  inflating: train/xray_71929.png    
  inflating: train/xray_71930.png    
  inflating: train/xray_71931.png    
  inflating: train/xray_71932.png    
  inflating: train/xray_71933.png    
  inflating: train/xray_71934.png    
  inflating: train/xray_71935.png    
  inflating: train/xray_71936.png    
  inflating: train/xray_71937.png    

In [ ]:
import os

train_img_dir = "/content/train"

actual_images = len([
    f for f in os.listdir(train_img_dir)
    if f.endswith(".png")
])

print("Actual images:", actual_images)


Actual images: 76913


In [ ]:
# How many should be there
import json

train_json_path = "/content/drive/MyDrive/Datasets/PIDray_raw/annotations/train.json"

with open(train_json_path, "r") as f:
    data = json.load(f)

expected_images = len(data["images"])
print("Expected images:", expected_images)


Expected images: 76913


In [ ]:
# whats missing
image_files = set(os.listdir(train_img_dir))
json_files = set(img["file_name"] for img in data["images"])

missing = json_files - image_files
extra = image_files - json_files

print("Missing from Drive:", len(missing))
print("Extra in Drive:", len(extra))
print(extra)
print(sorted(list(missing))[:10])
print(sorted(list(missing))[-10::])
print(len(image_files))


Missing from Drive: 0
Extra in Drive: 0
set()
[]
[]
76913


TRAIN TEST SPLIT


In [ ]:
import json
import random

# Load train.json
with open("/content/drive/MyDrive/Datasets/PIDray_raw/annotations/train.json") as f:
    data = json.load(f)

images = data["images"]
annotations = data["annotations"]

# Shuffle images
random.seed(42)
random.shuffle(images)

split_idx = int(len(images) * 0.9)

train_images = images[:split_idx]
val_images = images[split_idx:]

train_ids = set(img["id"] for img in train_images)
val_ids = set(img["id"] for img in val_images)

train_annotations = [ann for ann in annotations if ann["image_id"] in train_ids]
val_annotations = [ann for ann in annotations if ann["image_id"] in val_ids]

print("Train images:", len(train_images))
print("Val images:", len(val_images))


Train images: 69221
Val images: 7692


COCO -> YOLO

In [ ]:
import os
import shutil
from collections import defaultdict

IMG_ROOT = "/content/train"
OUTPUT_ROOT = "/content/PIDray_YOLO"

os.makedirs(f"{OUTPUT_ROOT}/images/train", exist_ok=True)
os.makedirs(f"{OUTPUT_ROOT}/images/val", exist_ok=True)
os.makedirs(f"{OUTPUT_ROOT}/labels/train", exist_ok=True)
os.makedirs(f"{OUTPUT_ROOT}/labels/val", exist_ok=True)

def convert_to_yolo(images_list, annotations_list, split):
    img_lookup = {img["id"]: img for img in images_list}

    ann_by_image = defaultdict(list)
    for ann in annotations_list:
        ann_by_image[ann["image_id"]].append(ann)

    for img_id, img in img_lookup.items():
        file_name = img["file_name"]
        width = img["width"]
        height = img["height"]

        src_img_path = os.path.join(IMG_ROOT, file_name)
        dst_img_path = os.path.join(OUTPUT_ROOT, f"images/{split}", file_name)

        if not os.path.exists(src_img_path):
            continue

        shutil.move(src_img_path, dst_img_path) # CHANGED FROM SHUTUTIL.COPY TO SHUTUTIL.MOVE

        label_path = os.path.join(OUTPUT_ROOT, f"labels/{split}", file_name.replace(".png", ".txt"))

        with open(label_path, "w") as f:
            for ann in ann_by_image[img_id]:
                x, y, w, h = ann["bbox"]

                x_center = (x + w / 2) / width
                y_center = (y + h / 2) / height
                w_norm = w / width
                h_norm = h / height

                class_id = ann["category_id"] - 1  # zero-index

                f.write(f"{class_id} {x_center} {y_center} {w_norm} {h_norm}\n")

convert_to_yolo(train_images, train_annotations, "train")
convert_to_yolo(val_images, val_annotations, "val")

print("Conversion complete.")


Conversion complete.


create pidray

In [ ]:
class_names = [cat["name"] for cat in data["categories"]]

yaml_content = f"""
path: /content/PIDray_YOLO
train: images/train
val: images/val

nc: {len(class_names)}
names: {class_names}
"""

with open("/content/PIDray_YOLO/pidray.yaml", "w") as f:
    f.write(yaml_content)

print("pidray.yaml created.")


pidray.yaml created.


ultralytics

In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.6 MB/s eta 0:00:00


LAST CHecks


In [ ]:
import random

print(len(os.listdir("/content/train")))
label_dir = "/content/PIDray_YOLO/labels/train"
label_val_dir = "/content/PIDray_YOLO/labels/val"
print(len(os.listdir(label_dir)))
print(len(os.listdir(label_val_dir)))


0
69221
7692


In [ ]:
import shutil
shutil.rmtree("/content/train")



In [ ]:
print(len(os.listdir("/content/train")))

0


In [ ]:
import random, os

sample = random.choice(os.listdir("/content/PIDray_YOLO/labels/train"))
print(sample)
print(open("/content/PIDray_YOLO/labels/train/" + sample).read())


xray_63475.txt



In [ ]:
# CHekc for negative

train_label_dir = "/content/PIDray_YOLO/labels/train"

empty = 0
non_empty = 0

for f in os.listdir(train_label_dir):
    if os.path.getsize(os.path.join(train_label_dir, f)) == 0:
        empty += 1
    else:
        non_empty += 1

print("Empty (negatives):", empty)
print("Non-empty (positives):", non_empty)


Empty (negatives): 42669
Non-empty (positives): 26552


train

yolo task=detect mode=train \
model=yolov8s.pt \
data=/content/PIDray_YOLO/pidray.yaml \
epochs=50 \
imgsz=640 \
batch=16


In [ ]:
# check if cuda is available

import torch
print(torch.cuda.is_available())


True


In [ ]:
from ultralytics import YOLO

# Load a model

model = YOLO("yolov8s.pt")

model.train(
    data="/content/PIDray_YOLO/pidray.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    device='0',
    project="/content/drive/MyDrive/PIDray_runs",
    name="detect_v1",
    exist_ok=True,
    save_period=1
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/PIDray_YOLO/pidray.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int